In [1]:
# Import necessary libraries
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error
from sklearn.preprocessing import LabelEncoder
from xgboost import XGBRegressor
from lightgbm import LGBMRegressor
from sklearn.feature_selection import SelectKBest,f_regression
from sklearn.ensemble import RandomForestRegressor
from catboost import CatBoostRegressor

In [2]:
df = pd.read_csv('D:/UNI WORK/kaggle_2/train.csv')

In [3]:
df.drop_duplicates(inplace=True)
df = df.drop(columns=['sub_area'], axis=1)

In [4]:
X = df.loc[:, df.columns != 'price_doc']
y = df[['price_doc']]

In [5]:
object_cols = X.select_dtypes(include=['object']).columns

In [6]:
# label = LabelEncoder()
# for col in object_cols:
#     X[col] = label.fit_transform(X[col].astype(str))
object_columns = X.select_dtypes(include=['object']).columns
X = pd.get_dummies(X, columns=object_columns, drop_first=True)

In [7]:
X=np.log1p(X)

In [8]:
num_cols = X.select_dtypes(include=['int64','float64']).columns

In [9]:
from sklearn.preprocessing import StandardScaler,MinMaxScaler,RobustScaler

scaler = RobustScaler()
X[num_cols] = scaler.fit_transform(X[num_cols])
X.shape

(178661, 273)

In [10]:
# from sklearn.decomposition import PCA
# pca = PCA(n_components=50)
# X = pca.fit_transform(X)


# skb = SelectKBest(f_regression, k=25)
# X = skb.fit_transform(X, y)
# X.shape

In [11]:
# X = X[['full_sq', 'life_sq', 'floor', 'school_education_centers_top_20_raion',
#        '16_29_female', 'build_count_monolith', 'industrial_km',
#        'church_synagogue_km', 'prom_part_500', 'trc_sqm_500',
#        'cafe_count_500_price_1000', 'cafe_count_500_price_high',
#        'mosque_count_500', 'leisure_count_500', 'green_part_1000',
#        'office_sqm_1000', 'cafe_count_1000_price_high', 'mosque_count_1000',
#        'leisure_count_1000', 'market_count_1000', 'trc_sqm_2000',
#        'sport_count_2000', 'incineration_raion_yes',
#        'nuclear_reactor_raion_yes', 'big_road1_1line_yes']]


In [12]:
df_test = pd.read_csv('D:/UNI WORK/kaggle_2/test.csv')

In [13]:
df_test = df_test.drop(['row ID'], axis=1)
df_test = df_test.drop(columns=['sub_area'], axis=1)

In [14]:
# label_test = LabelEncoder()
# for col in object_cols:
#     df_test[col] = label_test.fit_transform(df_test[col].astype(str))
object_columns = df_test.select_dtypes(include=['object']).columns
df_test = pd.get_dummies(df_test, columns=object_columns, drop_first=True)

In [15]:
df_test=np.log1p(df_test)

In [16]:
df_test[num_cols] = scaler.transform(df_test[num_cols])


In [17]:
# df_test = pca.transform(df_test)

In [18]:
# df_test = skb.transform(df_test)

In [19]:

# df_test = df_test[['full_sq', 'life_sq', 'floor', 'school_education_centers_top_20_raion',
#        '16_29_female', 'build_count_monolith', 'industrial_km',
#        'church_synagogue_km', 'prom_part_500', 'trc_sqm_500',
#        'cafe_count_500_price_1000', 'cafe_count_500_price_high',
#        'mosque_count_500', 'leisure_count_500', 'green_part_1000',
#        'office_sqm_1000', 'cafe_count_1000_price_high', 'mosque_count_1000',
#        'leisure_count_1000', 'market_count_1000', 'trc_sqm_2000',
#        'sport_count_2000', 'incineration_raion_yes',
#        'nuclear_reactor_raion_yes', 'big_road1_1line_yes']]

In [20]:
X.shape

(178661, 273)

In [21]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [31]:
# Create an XGBRegressor model
model = XGBRegressor(
    objective='reg:squarederror',  # Specify regression as the objective
    n_estimators=1000,  # Number of boosting rounds
    learning_rate=0.1,  # Step size shrinkage
    max_depth=6,
     n_jobs=-1,
      early_stopping_rounds=3,
     subsample=0.7, colsample_bytree=0.8 # Maximum depth of a tree
)
# model = LGBMRegressor(
#     objective='regression',
#     metric='rmse',
#     num_leaves=215,
#     learning_rate=0.1,
#     feature_fraction=0.9,n_estimators=215  ,max_depth=12 
    
# )
# model = RandomForestRegressor(
#     n_estimators=100,  # Number of trees in the forest
#     max_depth=10,     # Maximum depth of the tree
#     random_state=42 ,n_jobs=-1,verbose=2,warm_start=True
# )
# model =  CatBoostRegressor(iterations=1000, 
#                            depth=8, learning_rate=0.05,
#                              loss_function='RMSE')


# Fit the model to the training data
model.fit(X, y)


KeyboardInterrupt



In [29]:
y_pred = model.predict(df_test)

In [24]:
# rmse = np.sqrt(mean_squared_error(y_test, y_pred))
# print(f'Root Mean Squared Error (RMSE): {rmse}')

In [30]:
df = pd.DataFrame(y_pred)
custom_starting_index = 1
df.insert(0, 'RecordID', range(custom_starting_index, custom_starting_index + len(df)))

custom_column_names = ['row ID', 'price_doc']
csv_file_path = 'XgboostRegressor.csv'

df.to_csv(csv_file_path, index=False, header=custom_column_names)